# Importing requirements 

In [71]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen as uReq
import lxml
from pandas import DataFrame


In [72]:

wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text


soup = BeautifulSoup(raw_wikipedia_page,'xml')

# Extracting table from wikipedia

In [73]:
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# print(table)

for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
        if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'):
            
            continue
            
           
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            
            continue
            
            
    except:
        
        pass
    
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        
        continue
        
        
        
        
        
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

# Integrating Postcodes

In [74]:
unique_p = set(Postcode)
print('Number of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

Number of unique Postal codes: 77


# Creating the Data Frame 

In [75]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head(14)


,Postcode,Borough,Neighbourhood
0,M1N,Scarborough,Birch Cliff
1,M4B,East York,"Woodbine Gardens, Parkview Hill"
2,M1B,Scarborough,"Rouge, Malvern"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M9C,Etobicoke,Markland Wood
5,M4J,East York,East Toronto
6,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre"
7,M4L,East Toronto,India Bazaar
8,M1X,Scarborough,Upper Rouge
9,M4X,Downtown Toronto,"Cabbagetown, St. James Town"


In [76]:
df_toronto.shape

(77, 3)

In [77]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # for address conversion


#this conda install might take some time it took me about 20 minutes for the environment to be installed 


Solving environment: done

# All requested packages already installed.



# Processing as CSV 

In [78]:
url='http://cocl.us/Geospatial_data'
df_pcodes=pd.read_csv(url)
df_pcodes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Organizing Data Frame 

In [79]:
df_pcodes.columns = ['Postcode', 'Latitude', 'Longitude']
df_pcodes.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [80]:
df2 = df_toronto
df2.head

<bound method NDFrame.head of    Postcode           Borough  \
0       M1N       Scarborough   
1       M4B         East York   
2       M1B       Scarborough   
3       M6A        North York   
4       M9C         Etobicoke   
5       M4J         East York   
6       M5K  Downtown Toronto   
7       M4L      East Toronto   
8       M1X       Scarborough   
9       M4X  Downtown Toronto   
10      M4G         East York   
11      M5P   Central Toronto   
12      M1T       Scarborough   
13      M4K      East Toronto   
14      M8Y         Etobicoke   
15      M8V         Etobicoke   
16      M4A        North York   
17      M6L        North York   
18      M2M        North York   
19      M5M        North York   
20      M5R   Central Toronto   
21      M9V         Etobicoke   
22      M1V       Scarborough   
23      M6R      West Toronto   
24      M2K        North York   
25      M3K        North York   
26      M6S      West Toronto   
27      M8W         Etobicoke   
28      M5A  

# Sorting by Postcode

In [81]:
df2s=df2.sort_values('Postcode')
df2s.head()

,Postcode,Borough,Neighbourhood
2,M1B,Scarborough,"Rouge, Malvern"
61,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
55,M1E,Scarborough,"Morningside, West Hill"
76,M1G,Scarborough,Woburn
51,M1J,Scarborough,Scarborough Village


# 103 Rows by 5 Columns Requested 

In [82]:
neighborhoods=pd.merge(df2s,df_pcodes, how='right', on = 'Postcode')
neighborhoods.head(15)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
5,M1K,Scarborough,"Ionview, Kennedy Park",43.727929,-79.262029
6,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
7,M1M,Scarborough,"Cliffcrest, Cliffside",43.716316,-79.239476
8,M1N,Scarborough,Birch Cliff,43.692657,-79.264848
9,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304


In [84]:
neighborhoods.shape

(103, 5)